In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
# PyCaret 라이브러리 임포트
from pycaret.classification import *

# 데이터 로드
print("데이터 로드 중...")
train_df = pd.read_csv('../train.csv')
test_df = pd.read_csv('../test.csv')
sample_submission = pd.read_csv('../sample_submission.csv')
print(f"학습 데이터 크기: {train_df.shape}")
print(f"테스트 데이터 크기: {test_df.shape}")
print(f"제출 샘플 크기: {sample_submission.shape}")

# 기본적인 데이터 정보 확인
print("\n학습 데이터 정보:")
print(train_df.info())
print("\n결측치 확인:")
print(train_df.isnull().sum())

# 타겟 변수 분포 확인
print("\n타겟 변수(Cancer) 분포:")
print(train_df['Cancer'].value_counts())
print(f"암 양성 비율: {train_df['Cancer'].mean() * 100:.2f}%")

# 1. 데이터 전처리
print("\n데이터 전처리 진행 중...")
# ID 열 drop
train_df_processed = train_df.drop('ID', axis=1)
test_df_processed = test_df.drop('ID', axis=1)

# 범주형(문자열) 변수와 수치형 변수 분리
categorical_cols = train_df_processed.select_dtypes(include=['object']).columns
numerical_cols = train_df_processed.select_dtypes(exclude=['object']).columns.drop('Cancer')
print(f"\n범주형 변수: {list(categorical_cols)}")
print(f"수치형 변수: {list(numerical_cols)}")

# 2. 파생 변수 생성
print("\n특성 엔지니어링 진행 중...")

# ----- 아래 코드 추가 (요청한 파생 변수 생성) -----

# 갑상선 호르몬 정상 범위 변수 생성
train_df_processed['TSH_Normal'] = ((train_df_processed['TSH_Result'] >= 0.4) & 
                                    (train_df_processed['TSH_Result'] <= 4.0)).astype(int)
test_df_processed['TSH_Normal'] = ((test_df_processed['TSH_Result'] >= 0.4) & 
                                   (test_df_processed['TSH_Result'] <= 4.0)).astype(int)

train_df_processed['T4_Normal'] = ((train_df_processed['T4_Result'] >= 4.5) & 
                                   (train_df_processed['T4_Result'] <= 11.2)).astype(int)
test_df_processed['T4_Normal'] = ((test_df_processed['T4_Result'] >= 4.5) & 
                                  (test_df_processed['T4_Result'] <= 11.2)).astype(int)

train_df_processed['T3_Normal'] = ((train_df_processed['T3_Result'] >= 0.8) & 
                                   (train_df_processed['T3_Result'] <= 2.0)).astype(int)
test_df_processed['T3_Normal'] = ((test_df_processed['T3_Result'] >= 0.8) & 
                                  (test_df_processed['T3_Result'] <= 2.0)).astype(int)

# 호르몬 비율 변수 생성
train_df_processed['TSH_T4_Ratio'] = train_df_processed['TSH_Result'] / train_df_processed['T4_Result']
test_df_processed['TSH_T4_Ratio'] = test_df_processed['TSH_Result'] / test_df_processed['T4_Result']

train_df_processed['T3_T4_Ratio'] = train_df_processed['T3_Result'] / train_df_processed['T4_Result']
test_df_processed['T3_T4_Ratio'] = test_df_processed['T3_Result'] / test_df_processed['T4_Result']

# 복합 위험 점수 계산을 위한 이진 변환
binary_mappings = {
    'Family_Background': {'Negative': 0, 'Positive': 1},
    'Radiation_History': {'Unexposed': 0, 'Exposed': 1},
    'Smoke': {'Non-Smoker': 0, 'Smoker': 1},
    'Weight_Risk': {'Not Obese': 0, 'Obese': 1},
    'Diabetes': {'No': 0, 'Yes': 1}
}

# 이진 범주형 변수 변환 - 모델 학습용
for col, mapping in binary_mappings.items():
    train_df_processed[f'{col}_Binary'] = train_df_processed[col].map(mapping)
    test_df_processed[f'{col}_Binary'] = test_df_processed[col].map(mapping)

# 복합 위험 점수 생성
train_df_processed['Risk_Score'] = (train_df_processed['Family_Background_Binary'] + 
                                    train_df_processed['Radiation_History_Binary'] + 
                                    train_df_processed['Smoke_Binary'] + 
                                    train_df_processed['Weight_Risk_Binary'] + 
                                    train_df_processed['Diabetes_Binary'])
test_df_processed['Risk_Score'] = (test_df_processed['Family_Background_Binary'] + 
                                   test_df_processed['Radiation_History_Binary'] + 
                                   test_df_processed['Smoke_Binary'] + 
                                   test_df_processed['Weight_Risk_Binary'] + 
                                   test_df_processed['Diabetes_Binary'])

# 연령 및 성별 상호작용
train_df_processed['Gender_Numeric'] = train_df_processed['Gender'].map({'M': 0, 'F': 1})
test_df_processed['Gender_Numeric'] = test_df_processed['Gender'].map({'M': 0, 'F': 1})

train_df_processed['Age_Gender_Interaction'] = train_df_processed['Age'] * train_df_processed['Gender_Numeric']
test_df_processed['Age_Gender_Interaction'] = test_df_processed['Age'] * test_df_processed['Gender_Numeric']

# 결절 크기와 호르몬 상호작용
train_df_processed['Nodule_TSH_Interaction'] = train_df_processed['Nodule_Size'] * train_df_processed['TSH_Result']
test_df_processed['Nodule_TSH_Interaction'] = test_df_processed['Nodule_Size'] * test_df_processed['TSH_Result']

# ----- 여기까지 추가 -----

# 기존 코드 (아이오딘 결핍과 국가 상호작용 특성 생성)
train_df_processed['Iodine_Country_Risk'] = ((train_df_processed['Iodine_Deficiency'] == 'Deficient') & 
                                            (train_df_processed['Country'] == 'IND')).astype(int)
test_df_processed['Iodine_Country_Risk'] = ((test_df_processed['Iodine_Deficiency'] == 'Deficient') & 
                                           (test_df_processed['Country'] == 'IND')).astype(int)

# 수치형 변수의 이상치 클리핑
# 새로 추가된 수치형 변수들을 numerical_cols에 추가
new_numerical_cols = [
    'Iodine_Country_Risk', 'TSH_Normal', 'T4_Normal', 'T3_Normal', 
    'TSH_T4_Ratio', 'T3_T4_Ratio',
    'Family_Background_Binary', 'Radiation_History_Binary', 
    'Smoke_Binary', 'Weight_Risk_Binary', 'Diabetes_Binary',
    'Risk_Score', 'Gender_Numeric', 'Age_Gender_Interaction', 
    'Nodule_TSH_Interaction'
]

# 기존 numerical_cols에 새 변수들 추가 (이진 변수는 스케일링 제외)
scaling_cols = list(numerical_cols) + ['TSH_T4_Ratio', 'T3_T4_Ratio', 'Risk_Score', 
                                      'Age_Gender_Interaction', 'Nodule_TSH_Interaction']

# 원래 수치형 변수에 대해서만 이상치 클리핑 수행
for col in numerical_cols:
    q1 = train_df_processed[col].quantile(0.01)
    q3 = train_df_processed[col].quantile(0.99)
    train_df_processed[col] = train_df_processed[col].clip(q1, q3)
    test_df_processed[col] = test_df_processed[col].clip(q1, q3)

# 새로 생성된 비율 변수들에 대해서도 이상치 클리핑 수행
for col in ['TSH_T4_Ratio', 'T3_T4_Ratio', 'Age_Gender_Interaction', 'Nodule_TSH_Interaction']:
    if col in train_df_processed.columns:
        q1 = train_df_processed[col].quantile(0.01)
        q3 = train_df_processed[col].quantile(0.99)
        train_df_processed[col] = train_df_processed[col].clip(q1, q3)
        test_df_processed[col] = test_df_processed[col].clip(q1, q3)

# StandardScaler 적용 (스케일링이 필요한 수치형 변수들에만 적용)
print("\nStandardScaler 적용 중...")
scaler = StandardScaler()
train_df_processed[scaling_cols] = scaler.fit_transform(train_df_processed[scaling_cols])
test_df_processed[scaling_cols] = scaler.transform(test_df_processed[scaling_cols])

# 학습 데이터와 검증 데이터 분리 (20%)
train_data, val_data = train_test_split(train_df_processed, test_size=0.2, random_state=42, stratify=train_df_processed['Cancer'])
print(f"학습 데이터 크기: {train_data.shape}")
print(f"검증 데이터 크기: {val_data.shape}")

# 3. PyCaret 설정 및 모델 비교
print("\nPyCaret으로 여러 모델 비교 중...")

# 모든 수치형 변수 목록 업데이트 (기존 + 새로 생성된 변수들)
all_numeric_features = list(numerical_cols) + new_numerical_cols

# PyCaret 세션 초기화
clf = setup(
    data=train_data,  # 학습 데이터
    target='Cancer',  # 타겟 변수
    numeric_features=all_numeric_features,  # 수치형 변수
    categorical_features=list(categorical_cols),  # 범주형 변수
    ignore_features=['ID'] if 'ID' in train_data.columns else [],  # 무시할 변수
    normalize=False,  # 이미 StandardScaler 적용했으므로 False로 설정
    transformation=True,  # 데이터 변환 수행
    fix_imbalance=True,  # 클래스 불균형 처리
    session_id=42,  # 랜덤 시드
    fold=5,  # 5-폴드 교차 검증
    verbose=True  # 세부 정보 출력
)

# 모든 모델 비교 - 기본 설정으로 모든 모델 학습 및 평가
print("\n모든 모델 학습 및 비교 중...")
best_models = compare_models(
    sort='f1',  # F1 점수 기준으로 정렬
    n_select=3,  # 상위 3개 모델 선택
    verbose=True  # 세부 정보 출력
)

# 여러 개의 모델이 있을 경우 첫 번째(가장 좋은) 모델 선택
if isinstance(best_models, list):
    best_model = best_models[0]
else:
    best_model = best_models

# 최고 모델 이름 출력
print(f"\n최고 성능 모델: {best_model}")

# 모델 분석
print("\n최고 모델 분석 중...")
# 학습된 모델 평가
evaluate_model(best_model)

# 하이퍼파라미터 튜닝
print("\n최고 모델 하이퍼파라미터 튜닝 중...")
tuned_model = tune_model(
    best_model,
    optimize='f1',  # F1 점수 최적화
    n_iter=10,  # 10번의 튜닝 반복
    verbose=True  # 세부 정보 출력
)

# 튜닝된 모델 평가
print("\n튜닝된 모델 평가 중...")
evaluate_model(tuned_model)

# 모델 해석
print("\n모델 해석 중...")
try:
    # 기본 해석 방법 사용 (SHAP 없이)
    interpret_model(tuned_model)
except Exception as e:
    print(f"모델 해석 중 오류 발생: {e}")
    print("모델 해석을 건너뛰고 계속 진행합니다.")

# 최종 모델 학습
print("\n최종 모델 생성 중...")
final_model = finalize_model(tuned_model)

# 최종 모델 저장
print("\n최종 모델 저장 중...")
save_model(final_model, 'final_cancer_model')

# 테스트 데이터에 예측
print("\n테스트 데이터에 예측 중...")
predictions = predict_model(final_model, data=test_df_processed)

# 예측 결과 추출
test_predictions = predictions['prediction_label'].astype(int)

# sample_submission 파일에 예측 결과 저장
sample_submission['Cancer'] = test_predictions
sample_submission.to_csv('pycaret_submission.csv', index=False)
print("\n예측 완료! 'pycaret_submission.csv' 파일에 결과가 저장되었습니다.")

# 예측 결과 확인
print("예측 결과 분포:")
print(sample_submission['Cancer'].value_counts())
print(f"암 양성 예측 비율: {sample_submission['Cancer'].mean() * 100:.2f}%")

# 상위 3개 모델을 앙상블하여 성능 향상 시도
if isinstance(best_models, list) and len(best_models) > 1:
    print("\n상위 모델 앙상블 시도...")
    ensemble_model = blend_models(best_models, optimize='f1')
    
    # 앙상블 모델 평가
    evaluate_model(ensemble_model)
    
    # 앙상블 모델로 예측
    ensemble_predictions = predict_model(ensemble_model, data=test_df_processed)
    ensemble_test_predictions = ensemble_predictions['prediction_label'].astype(int)
    
    # 앙상블 결과 저장
    sample_submission['Cancer'] = ensemble_test_predictions
    sample_submission.to_csv('ensemble_submission.csv', index=False)
    
    print("\n앙상블 예측 완료! 'ensemble_submission.csv' 파일에 결과가 저장되었습니다.")
    print("앙상블 예측 결과 분포:")
    print(sample_submission['Cancer'].value_counts())
    print(f"암 양성 예측 비율: {ensemble_test_predictions.mean() * 100:.2f}%")

# 성능 비교를 위한 모델별 중요 지표 시각화
print("\n모델별 성능 지표 시각화...")
plot_model(best_model, plot='confusion_matrix')
plot_model(best_model, plot='auc')
plot_model(best_model, plot='feature')
print("\n분석 완료!")

데이터 로드 중...
학습 데이터 크기: (87159, 16)
테스트 데이터 크기: (46204, 15)
제출 샘플 크기: (46204, 2)

학습 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87159 entries, 0 to 87158
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 87159 non-null  object 
 1   Age                87159 non-null  int64  
 2   Gender             87159 non-null  object 
 3   Country            87159 non-null  object 
 4   Race               87159 non-null  object 
 5   Family_Background  87159 non-null  object 
 6   Radiation_History  87159 non-null  object 
 7   Iodine_Deficiency  87159 non-null  object 
 8   Smoke              87159 non-null  object 
 9   Weight_Risk        87159 non-null  object 
 10  Diabetes           87159 non-null  object 
 11  Nodule_Size        87159 non-null  float64
 12  TSH_Result         87159 non-null  float64
 13  T4_Result          87159 non-null  float64
 14  T3_Result          87159 n

,Description,Value
0,Session id,42
1,Target,Cancer
2,Target type,Binary
3,Original data shape,"(69727, 30)"
4,Transformed data shape,"(106821, 43)"
5,Transformed train set shape,"(85902, 43)"
6,Transformed test set shape,"(20919, 43)"
7,Numeric features,20
8,Categorical features,9
9,Preprocess,True



모든 모델 학습 및 비교 중...


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8845,0.7046,0.4280,0.5228,0.4705,0.4065,0.4091,4.6800
lightgbm,Light Gradient Boosting Machine,0.8839,0.7054,0.4094,0.5205,0.4581,0.3942,0.3979,0.9460
catboost,CatBoost Classifier,0.8840,0.7018,0.3406,0.5252,0.4131,0.3520,0.3622,4.1720
rf,Random Forest Classifier,0.8824,0.7022,0.3445,0.5150,0.4128,0.3504,0.3591,1.6000
et,Extra Trees Classifier,0.8797,0.6954,0.3224,0.4980,0.3913,0.3280,0.3376,1.2320
ada,Ada Boost Classifier,0.8673,0.6856,0.3143,0.4320,0.3614,0.2898,0.2955,1.2160
nb,Naive Bayes,0.7733,0.6834,0.5289,0.2718,0.3590,0.2382,0.2578,0.3100
ridge,Ridge Classifier,0.8772,0.6707,0.2351,0.4762,0.3147,0.2557,0.2751,0.3320
lr,Logistic Regression,0.6837,0.6608,0.5556,0.2025,0.2966,0.1465,0.1777,1.4520
dt,Decision Tree Classifier,0.8077,0.5920,0.3082,0.2529,0.2778,0.1681,0.1692,0.7100



최고 성능 모델: GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

최고 모델 분석 중...


...


최고 모델 하이퍼파라미터 튜닝 중...


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8855,0.7026,0.4637,0.5257,0.4927,0.4285,0.4295
1,0.8819,0.6928,0.4548,0.5091,0.4804,0.4140,0.4148
2,0.8811,0.6889,0.4275,0.5055,0.4632,0.3969,0.3987
3,0.8873,0.7138,0.4936,0.5327,0.5124,0.4488,0.4492
4,0.8867,0.7128,0.4808,0.5306,0.5045,0.4407,0.4414
Mean,0.8845,0.7022,0.4641,0.5207,0.4907,0.4258,0.4267
Std,0.0025,0.0101,0.0227,0.0112,0.0175,0.0186,0.0182


Fitting 5 folds for each of 10 candidates, totalling 50 fits

튜닝된 모델 평가 중...


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…


모델 해석 중...
모델 해석 중 오류 발생: This function only supports tree based models for binary classification: dt, rf, lightgbm, et, catboost.
모델 해석을 건너뛰고 계속 진행합니다.

최종 모델 생성 중...
